In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GitHub" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect


NOTE: User is responsible for checking the content of datasets and the applicable licenses and determining if suitable for the intended use.
"""
# If you're using Google Colab and not running locally, run this cell to install dependencies


# Install dependencies
!pip install wget
!apt-get update && apt-get install -y sox libsndfile1 ffmpeg
!pip install text-unidecode
!pip install omegaconf

BRANCH='main'

!python -m pip install git+https://github.com/NVIDIA/NeMo.git@{BRANCH}#egg=nemo_toolkit[asr]

In [ ]:
# import libraries

import glob
import json
import librosa
import numpy as np
from omegaconf import OmegaConf, open_dict
import os
import soundfile as sf
import subprocess
import tarfile
import tqdm
import wget
import re

import torch

# Introduction to Canary models
Canary is a family of multilingual, multitask speech-to-text models based on the attention encoder-decoder (AED) architecture. All Canary models use a FastConformer encoder and Transformer decoder. The current lineup includes `canary-1b-v2`, `canary-1b`, `canary-1b-flash`, and `canary-180m-flash`.

`canary-1b-v2` is the latest and most comprehensive model, supporting speech recognition for 25 European languages, as well as translation between English and these languages (En<->X). It introduces new features such as parallel chunking and full timestamp prediction across all supported languages.

`canary-1b-flash` (883M parameters) and canary-180m-flash (182M parameters) are optimized for speed and efficiency. These models support speech recognition in English, German, French, and Spanish, as well as translation between English and German/French/Spanish (in both directions). They also offer output with or without punctuation and capitalization (PnC), and support word-level timestamp prediction for all four languages. The canary-1b-flash model achieves faster and more accurate results than `canary-1b` by increasing the encoder size and reducing the decoder size, improving speed while maintaining comparable model capacity.
In this tutorial, we will focus primarily on the Canary-1b-v2 model.
Refer to the following resources for more details:

🤗[canary-1b-v2](https://huggingface.co/nvidia/canary-1b-v2)

🤗[canary-1b](https://huggingface.co/nvidia/canary-1b)

🤗[canary-1b-flash](https://huggingface.co/nvidia/canary-1b-flash)

🤗[canary-180m-flash](https://huggingface.co/nvidia/canary-180m-flash)

[Canary-1B paper](https://arxiv.org/abs/2406.19674)

[Canary-1B-flash paper](https://arxiv.org/abs/2503.05931)





## Components of Canary architecture

### Model architecture

The input audio is converted into 128-dim log-mel features extracted for 25ms window with a stride of 10ms. The spectrogram features are then processed through the encoder. The decoder conditions on the encoder output and decoder prompt to autoregressively generate one token at a time.

<img src="images/promptformat.png" width="750" height="400">

### Decoder prompt

Decoder prompt is the key to attaining multitask capability with Canary models. Decoder prompt is a sequence of special tokens that define the precise task (language output text, punctuations, timestamps, etc.) to be performed on the input audio.
As shown in the figure, the decoder takes a sequence of prompt tokens as input before generating output text. The example prompt sequence corresponds to English speech recognition as the language for input audio and output text is set to English. The format of the decoder prompt is defined by `TEMPLATE["user"]["template"]` in the [Canary2PromptFormatter](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/common/prompts/canary2.py).


### Tokenizers
<img src="images/tokenizer.png" width="600" height="400">


For Canary-1b-v2, we use a unified SentencePiece [tokenizer](https://github.com/NVIDIA/NeMo/blob/main/scripts/tokenizers/process_asr_text_tokenizer.py).

For all other Canary models, we use the concatenated [tokenizer](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/common/tokenizers/canary_tokenizer.py), which combines language-specific SentencePiece tokenizers with shared special tokens. Each language uses a vocabulary of 1024 subword tokens, and these per-language vocabularies are concatenated together as shown in the figure below.

In addition to language-specific tokens, Canary uses 1152 tokens to represent special tokens. Special tokens include generic tokens such as `<|startoftranscript|>`, `<|endoftext|>`, `<pad>`, as well as many other task-specific tokens.
Listed below is a variety of special tokens that the default tokenizer includes. This should give an idea of various tasks that can be supported with the current tokenizer and prompt formatter.

* Task-specific tokens; these provide a control for tasks and output characteristics, such as decoding with or without punctuations and capitalizations (`<|pnc|>` or `<|nopnc|>`), timestamp prediction (`<|timestamp|>` or `<|notimestamp|>`), emotion recognition (`<|emo:undefined|>`, `<|emo:neutral|>`, `<|emo:happy|>`, `<|emo:sad|>`, `<|emo:angry|>`).
* Language identity tokens; the default `spl_tokens` tokenizer supports 184 language IDs, including an `<|unklang|>` token. Language identity tokens are used to encode `source_lang`, `target_lang` fields in the manifest.
* Integer tokens; timestamp prediction uses integer values, between `0` and `899` to denote frame numbers corresponding to word start and word end.
* Speaker ID tokens; although current Canary-flash models are not trained for speaker identity, the default tokenizer includes 16 speaker ID tokens, `<|spk0|> ... <|spk15|>`.
* Additional tokens; the default tokenizer incldes 30 additional tokens, `<|spltoken0|> ... <|spltoken29|>`, not assigned to any perform any particular function. The user can use one of these to represent a custom behavior.


# Outline

The tutorial is divided into four sections.

First, we see how to perform inference using Canary models, specifically speech recognition, translation, and timestamp prediction.

Then we learn how to train a Canary model in two ways -- from scratch and from an initial checkpoint. We will train a model for English speech recognition.

Next, we look deeper into various use cases for Canary model with detailed guidelines on how to use Canary-style training for various tasks.

Finally, we share some practitioner's tips from our experience working with Canary models.

# Download LibriLight data
We download LibriLight data so we can run inference on audio samples. We'll later use the small 1 hour split for training a custom Canary model.

In [ ]:
def download_and_prepare_librilight_data(data_dir="datasets"):
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    libri_data_dir = os.path.join(data_dir, 'LibriLight')
    libri_tgz_file = f'{data_dir}/librispeech_finetuning.tgz'

    if not os.path.exists(libri_tgz_file):
        url = "https://dl.fbaipublicfiles.com/librilight/data/librispeech_finetuning.tgz"
        libri_path = wget.download(url, data_dir, bar=None)
        print(f"Dataset downloaded at: {libri_path}")

    if not os.path.exists(libri_data_dir):
        tar = tarfile.open(libri_tgz_file)
        tar.extractall(path=libri_data_dir)

    print(f'LibriLight data is ready at {libri_data_dir}')

download_and_prepare_librilight_data()

# Inference with Canary-1b-v2 model

We run inference on a sample audio files, both short and long, to demonstrate the various capabilities supported by the released Canary-1b-v2 checkpoints.

Canary inference uses the `trancribe` method of `EncDecMultiTaskModel`.
The user can control the task and language for the inference using specific arguments to `transcribe`. These arguments control the prompt token sequence passed as an input to the decoder (decoder prompt is discussed in more detail in the next section).

See examples below for using `transcribe` to perform various tasks.

In [ ]:
from pydub import AudioSegment
from IPython.display import Audio, display

def listen_to_audio(audio_path, offset=0.0, duration=-1):
    audio = AudioSegment.from_file(audio_path)
    start_ms = int(offset * 1000)
    if duration == -1:
        end_ms = -1
    else:
        end_ms = int((offset+duration) * 1000)

    segment = audio[start_ms:end_ms]
    audio = Audio(segment.export(format='wav').read())
    display(audio)

## Load model

Load the model of your choice.

We use `canary-1b-v2` in these inference examples.

If testing a local checkpoint, use the following code snippet in place of the one below:
```
canary_model = EncDecMultiTaskModel.restore_from(
        restore_path=ckpt_path,
        map_location=map_location,
    )
```
```

In [ ]:
from nemo.collections.asr.models import EncDecMultiTaskModel
map_location = 'cuda' if torch.cuda.is_available() else 'cpu'
canary_model = EncDecMultiTaskModel.from_pretrained('nvidia/canary-1b-v2', map_location=map_location)

## Speech-to-text recognition

Here we pass the `source_lang` (language of audio input) and `target_lang` (language of recognized text) as `en`. Thus, this performs english speech recognition.



In [ ]:
audio_path = "datasets/LibriLight/librispeech_finetuning/1h/0/clean/3526/175658/3526-175658-0000.flac"
listen_to_audio(audio_path)

# To transcribe in a particular language; this example is for English, but will work for each of 25 supported languages
transcript = canary_model.transcribe(
  audio=[audio_path],
  batch_size=1,
  source_lang='en',	# en: English, es: Spanish, fr: French, de: German
  target_lang='en',	# should be same as "source_lang" for 'asr'
)
print("\n\nEnglish speech recognition:")
print(f'  \"{transcript[0].text}\"')

## Speech-to-text translation

Here we pass the `source_lang` (language of audio input) as `en` and `target_lang` (language of transcription text) as `es`. Thus, this performs English to Spanish speech-to-text translation.

In [ ]:
transcript = canary_model.transcribe(
  audio=[audio_path],
  batch_size=1,
  source_lang='en',	# en: English, es: Spanish, fr: French, de: German
  target_lang='es',	# should be same as "source_lang" for 'asr'
)
print("\n\nSpeech to text translation form English to Spanish with punctuations and capitalizations:")
print(f'  \"{transcript[0].text}\"')


## Timestamp Generation Workflow

Currently timestamps generation for `canary-1b-v2` is done via 3 steps

1. The audio is passed through the Canary v2 model, which is an AED (Attention Encoder-Decoder) multi-task model. The output is a token sequence produced by the decoder of the model.

2. The same audio is passed through the Multi-lingual Parakeet CTC model. From this model, we obtain the log-probabilities matrix produced by the CTC decoder of the Parakeet model. 
    This matrix represents the (log) probability of every possible token for each time frame (80ms time windows for the given models).

3. Viterbi Decoding: Given the token sequence (from Canary v2) and the log-probability matrix (from multilingual Parakeet CTC), we perform Viterbi Decoding. The goal is to find the most likely sequence of predicted tokens aligned over time frames.

<img src="images/canary2_timestamps.png" width="1000" height="400">


## Timestamp prediction

Timestamp prediction is supported for all langauges and can be performed with timestamp prediction by passing `timestamps=True` argument.

In [ ]:
# To recognize with timestamps
transcript = canary_model.transcribe(
  audio=[audio_path],
  batch_size=1,
  source_lang='en',	# en: English or other supported language
  target_lang='en',	# should be same as "source_lang" for 'asr'
  timestamps=True
)
print("\n\nEnglish speech to text recognition with timestamp prediction:\n")

print(f'Predicted output: \n"{transcript[0].text}\"')

print('\nSegment level timestamps:')
for sample in transcript[0].timestamp['segment']:
    segment, start, end = sample['segment'], sample['start'], sample['end']
    print(f'{segment}')
    print(f'Segment start: {start:.2f}s')
    print(f'Segment end: {end:.2f}s\n')

print('\nWord level timestamps:')
for sample in transcript[0].timestamp['word']:
    word, start, end = sample['word'], sample['start'], sample['end']
    print(f'{word:<15}[{start:.2f}s, {end:.2f}s]')
    # listen_to_audio(audio_path, offset=start, duration=(end-start)) # uncomment to listen to word segments

## Inference with longform input

Canary models natively handle inputs up to ~40 seconds. For longer audio, the input is split into 30–40 s chunks (minimizing padding on the final chunk) and processed in parallel.

For recordings longer than one hour, processing occurs in consecutive hour‑long segments.

Outputs are seamlessly stitched to produce a single, continuous result.


### Create a longform audio sample


As LibriLight does not have a long duration audio, we'll first create one by stitching together all utterances from a story.




In [ ]:
# Creating a longform audio sample

def get_longform_audio_sample(data_dir="datasets"):
    libri_data_dir = os.path.join(data_dir, 'LibriLight')
    audio_paths = glob.glob(os.path.join(libri_data_dir, 'librispeech_finetuning/1h/0/clean/3526/175658/3526-175658-*.flac'))
    audio_paths.sort() # sort by the utterance IDs
    write_path = os.path.join(libri_data_dir, 'longform','-'.join(os.path.basename(audio_paths[0]).split('-')[:2])+'.wav')
    os.makedirs(os.path.dirname(write_path), exist_ok=True)
    longform_audio_data = []
    for audio_path in audio_paths:
        data, sr = librosa.load(audio_path, sr=16000)
        longform_audio_data.extend(data)
    sf.write(write_path, longform_audio_data, sr)
    minutes, seconds = divmod(len(longform_audio_data)/sr, 60)
    print(f'{int(minutes)} min {int(seconds)} sec audio file saved at {write_path}')
    return write_path

longform_audio_path = get_longform_audio_sample()
listen_to_audio(longform_audio_path)

### Longform inference without timestamps

`.transcribe()` will perform inference on the long audio file `datasets/LibriLight/longform/3526-175658.wav`, which is currently just the one file that we created above. Alternatively you can also pass a path to a manifest file. We will discuss manifest creation in the the next section..

In [ ]:
transcript = canary_model.transcribe(
  audio=[longform_audio_path],
  batch_size=1,
  source_lang='en',
  target_lang='en',
)

In [ ]:
def print_sentences_per_item(items):
    for i, item in enumerate(items, 1):
        text = item.text if hasattr(item, "text") else str(item)
        sentences = [s.strip() for s in re.split(r'(?<=[.!?])\s+', text) if s.strip()]
        print(f"--- Audio {i} ---")
        for s in sentences:
            print(s if s[-1] in ".!?" else s + ".")
        print()
print_sentences_per_item(transcript)

### Longform inference with timestamps

We run the same command as above with `timestamps=True`. 

In [ ]:
transcript = canary_model.transcribe(
  audio=[longform_audio_path],
  batch_size=1,
  source_lang='en',
  target_lang='en',
  timestamps=True,
)

In [ ]:
print('\nWord level timestamps:')
for sample in transcript[0].timestamp['word']:
    word, start, end = sample['word'], sample['start'], sample['end']
    print(f'{word:<15}[{start:.2f}s, {end:.2f}s]')

# Train a Canary model on custom data

Now we will see how to train a Canary model on a custom data. Later we discuss how we can incorporate more languages and tasks.

In this example we'll see two ways to train a Canary model on a 1 hour split of the LibriLight data:

1. A small, 2-layer encoder, 2-layer decoder, version of the model trained from scratch.

2. A 180M model initialized from `canary-180m-flash`.

Different components needed for training are passed as an yaml config file to the training script.

Next, we'll prepare the following components required to set up the training,

```
model.train_ds.manifest_filepath=$MANIFEST_PATH \
model.tokenizer.langs.en.dir="$LANG_TOKENIZER_DIR" \
model.tokenizer.langs.spl_tokens.dir="$SPL_TOKENIZER_DIR" \
model.prompt_format="canary2" \
```

In [ ]:
canary_model = EncDecMultiTaskModel.from_pretrained('nvidia/canary-180m-flash', map_location=map_location)

## Prepare manifest

We'll build manifest from 1 hour split of LibriLight data. The manifest file has a dictionary corresponding to each training sample, something like this:
```
manifest_sample = {
    "audio_filepath": audio_path,
    "duration": duration,
    "text": transcript,
    "target_lang": "en",
    "source_lang": "en",
    "pnc": "False"
}
```

The prepared manifest file will be saved at `datasets/LibriLight/train_manifest.json`.

In [ ]:
def build_manifest(data_root, manifest_path):
    transcript_list = glob.glob(os.path.join(data_root, 'LibriLight/librispeech_finetuning/1h/**/*.txt'), recursive=True)
    tot_duration = 0
    with open(manifest_path, 'w') as fout:
        pass # make sure a new file is created
    for transcript_path in tqdm.tqdm(transcript_list):
        with open(transcript_path, 'r') as fin:
            wav_dir = os.path.dirname(transcript_path)
            with open(manifest_path, 'a') as fout:
                for line in fin:
                    # Lines look like this:
                    # fileID transcript
                    file_id = line.strip().split(' ')[0]
                    audio_path = os.path.join(wav_dir, f'{file_id}.flac')

                    transcript = ' '.join(line.strip().split(' ')[1:]).lower()
                    transcript = transcript.strip()

                    duration = librosa.core.get_duration(path=audio_path)
                    tot_duration += duration
                    # Write the metadata to the manifest
                    metadata = {
                      "audio_filepath": audio_path,
                      "duration": duration,
                      "text": transcript,
                      "lang": "en",
                      "target_lang": "en",
                      "source_lang": "en",
                      "pnc": "False"
                    }
                    json.dump(metadata, fout)
                    fout.write('\n')
    print(f'\n{np.round(tot_duration/3600)} hour audio data ready for training')

data_dir = "datasets"
train_manifest = os.path.join(data_dir, 'LibriLight/train_manifest.json')
build_manifest(data_dir, train_manifest)
print(f"LibriLight train manifests created at {train_manifest}.")

## Build tokenizer


As described in the introduction, we now build a tokenizer for special tokens and for English text from the training data.

**Note** that you do not need to train a new tokenizer if you are initializing from Canary-flash models for a task and language that the default tokenizers already support. At the end of this tutorial we discuss some cases where you'd want to retrain the tokenizer and reinitialize the token embeddings.

### Build tokenizer for special *tokens*

The tokenizer will be saved at `tokenizers/spl_tokens`. See `tokenizers/spl_tokens/tokenizer.vocab` for a 1152-unit vocabulary of tokens.

In [ ]:
BRANCH='r2.5.0'
def wget_from_nemo(nemo_script_path, local_dir="scripts"):
    os.makedirs(local_dir, exist_ok=True)
    script_url = f"https://raw.githubusercontent.com/NVIDIA/NeMo/refs/heads/{BRANCH}/{nemo_script_path}"
    script_path = os.path.basename(nemo_script_path)
    if not os.path.exists(f"{local_dir}/{script_path}"):
        !wget -P {local_dir}/ {script_url}

In [ ]:
wget_from_nemo("scripts/speech_recognition/canary/build_canary_2_special_tokenizer.py")
output_dir = "tokenizers/spl_tokens"
!mkdir -p {output_dir}
!python scripts/build_canary_2_special_tokenizer.py {output_dir}

### Build language-specific tokenizer

The tokenizer will be saved at `tokenizers/en_libri1h_1024/tokenizer_spe_bpe_v1024`. See `tokenizer.vocab` for a 1024-unit vocabulary of tokens.

In [ ]:
wget_from_nemo('scripts/tokenizers/process_asr_text_tokenizer.py')
LANG='en'
DATA='libri1h'
VOCAB_SIZE=1024
OUT_DIR = f"tokenizers/{LANG}_{DATA}_{VOCAB_SIZE}"
manifest_path = os.path.join(data_dir, 'LibriLight', 'train_manifest.json')
train_text_path = os.path.join(data_dir, 'LibriLight', 'train_text.lst')
with open(manifest_path, "r") as f:
    data = [json.loads(line.strip()) for line in f.readlines()]
with open(train_text_path, "w") as f:
    for line in data:
        f.write(f"{line['text']}\n")

!python scripts/process_asr_text_tokenizer.py \
  --data_file={train_text_path} \
  --vocab_size={VOCAB_SIZE} \
  --data_root={OUT_DIR} \
  --tokenizer="spe" \
  --spe_type=bpe \
  --spe_character_coverage=1.0 \
  --no_lower_case \
  --log


## Prompt format

Canary-flash decoder generates output text conditioned on audio encoder representations and the decoder prompt. As described in the introduction, Canary-Flash models use [Canary2PromptFormatter](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/common/prompts/canary2.py), and so we set the `prompt_format` accordingly

```
model.prompt_format="canary2"
```

For the samples in our training data the decoder prompt will have the following sequence of special tokens,

`<|startofcontext|><|startoftranscript|><|emo:undefined|><|en|><|en|><|nopnc|><|noitn|><|notimestamp|><|nodiarize|>`

Note that source language and target language are set to `en` for English speech recognition **without** pnc (`<|nopnc|>`), timestamps (`<|notimestamp|>`), emotion recognition (`<|emo:undefined|>`), or diarization (`<|nodiarize|>`).

## Train Canary model from scratch

Now we have all the components needed to train. We download a local copy of the training script and the default config. We pass the data and tokenizers we prepared above.

The tokenizers are processed as follows with their language IDs as keys.

```
model:
  tokenizer:
    langs:
      spl_tokens: # special tokens model
        dir: "tokenizers/spl_tokens"
        type: bpe
      en: # English tokenizer
        dir: "tokenizers/en_libri1h_1024/tokenizer_spe_bpe_v1024"
        type: bpe
```

We now train a small Canary model with 2 FastConformer encoder layers and 2 Transformer decoder layers.

In [ ]:
wget_from_nemo('examples/asr/speech_multitask/speech_to_text_aed.py')
wget_from_nemo('examples/asr/conf/speech_multitask/fast-conformer_aed.yaml', 'config')

In [ ]:
MANIFEST = os.path.join("datasets", "LibriLight", 'train_manifest.json')
!HYDRA_FULL_ERROR=1 python scripts/speech_to_text_aed.py \
  --config-path="../config" \
  --config-name="fast-conformer_aed.yaml" \
  name="canary-small" \
  model.prompt_format="canary2" \
  model.train_ds.manifest_filepath={MANIFEST} \
  model.validation_ds.manifest_filepath={MANIFEST} \
  model.test_ds.manifest_filepath={MANIFEST} \
  model.tokenizer.langs.en.dir="tokenizers/en_libri1h_1024/tokenizer_spe_bpe_v1024" \
  model.tokenizer.langs.spl_tokens.dir="tokenizers/spl_tokens" \
  spl_tokens.model_dir="tokenizers/spl_tokens" \
  model.encoder.n_layers=2 \
  model.transf_decoder.config_dict.num_layers=2 \
  exp_manager.exp_dir="canary_results" \
  exp_manager.resume_ignore_no_checkpoint=true \
  trainer.max_steps=10 \
  trainer.log_every_n_steps=1

## Train Canary model from a Canary flash checkpoint (aka fine-tuning)

We will now train a Canary model initialized from the `canary-180m-flash` checkpoint; in effect finetuning the `canary-180m-flash` model. This is the same checkpoint that we used to run sample inference in the previous section.

```
init_from_pretrained_model: canary-180m-flash
```

For the sake of simplicity, we will retain the exact same model architecture as `canary-180m-flash`. You can choose to include and exclude certain layers and parameters from the initial checkpoint; we discuss these customizations in the next section.

### Build config

We'll update the base config that we use in the example above and save the new config as `config/canary-180m-flash-finetune`.

In [ ]:
# Load canary model if not previously loaded in this notebook instance
if 'canary_model' not in locals():
    canary_model = EncDecMultiTaskModel.from_pretrained('nvidia/canary-180m-flash')

base_model_cfg = OmegaConf.load("config/fast-conformer_aed.yaml")

In the training config, we should ensure compatibility with the pre-trained model.

1. Set initialization from `canary-180m-flash`.

In [ ]:
base_model_cfg['name'] = 'canary-180m-flash-finetune'
base_model_cfg.pop("init_from_nemo_model", None)
base_model_cfg['init_from_pretrained_model'] = "nvidia/canary-180m-flash"

2. Set path to the tokenizers from the pre-trained model, so as to ensure that the fine-tuning uses a compatible tokenization. The following command reads tokenizers from `canary_model` and saves the files at `canary_flash_tokenizers/{lang}` directories.

In [ ]:
canary_model.save_tokenizers('./canary_flash_tokenizers/')

In [ ]:
for lang in os.listdir('canary_flash_tokenizers'):
    base_model_cfg['model']['tokenizer']['langs'][lang] = {}
    base_model_cfg['model']['tokenizer']['langs'][lang]['dir'] = os.path.join('canary_flash_tokenizers', lang)
    base_model_cfg['model']['tokenizer']['langs'][lang]['type'] = 'bpe'
base_model_cfg['spl_tokens']['model_dir'] = os.path.join('canary_flash_tokenizers', "spl_tokens")

3. Ensure that the prompt format and relevant parameters match.

In [ ]:
base_model_cfg['model']['prompt_format'] = canary_model._cfg['prompt_format']
base_model_cfg['model']['prompt_defaults'] = canary_model._cfg['prompt_defaults']

4. Ensure that the model architecture matches.

In [ ]:
base_model_cfg['model']['model_defaults'] = canary_model._cfg['model_defaults']
base_model_cfg['model']['preprocessor'] = canary_model._cfg['preprocessor']
base_model_cfg['model']['encoder'] = canary_model._cfg['encoder']
base_model_cfg['model']['transf_decoder'] = canary_model._cfg['transf_decoder']
base_model_cfg['model']['transf_encoder'] = canary_model._cfg['transf_encoder']

### Launch training
Save config and launch training.

In [ ]:
cfg = OmegaConf.create(base_model_cfg)
with open("config/canary-180m-flash-finetune.yaml", "w") as f:
    OmegaConf.save(cfg, f)

In [ ]:
MANIFEST = os.path.join("datasets", "LibriLight", 'train_manifest.json')
!HYDRA_FULL_ERROR=1 python scripts/speech_to_text_aed.py \
  --config-path="../config" \
  --config-name="canary-180m-flash-finetune.yaml" \
  name="canary-180m-flash-finetune" \
  model.train_ds.manifest_filepath={MANIFEST} \
  model.validation_ds.manifest_filepath={MANIFEST} \
  model.test_ds.manifest_filepath={MANIFEST} \
  exp_manager.exp_dir="canary_results" \
  exp_manager.resume_ignore_no_checkpoint=true \
  trainer.max_steps=10 \
  trainer.log_every_n_steps=1

# Guidance for different implementation scenarios

You can use the Canary-style training to develop a model for most speech applications. We saw one generic example of training on custom data from scratch on English speech recognition. Here we discuss how to handle several other scenarios.


## 1. Speech-to-text recognition and translation

When creating the manifest, make sure to pass the appropriate `source_lang` and `target_lang` tokens for each data point.

You'll need language-specific tokenizers for each language. You can build the tokenizer as we saw in the previous section.

The default `spl_tokens` tokenizer, supports 183 language IDs. If you want to use a language not currently represented, you can rebuild the tokenizer with a new set of `spl_tokens` that includes your language of choice.

Finally, in the config add paths to different tokenizers with their language IDs as keys.

```
model:
  tokenizer:
    langs:
      spl_tokens: # special tokens model
        dir: "tokenizers/spl_tokens"
        type: bpe
      en: # English tokenizer (example, replace with whichever language you would like or add tokenizers to add tokenizer for additional languages)
        dir: "tokenizers/spe_bpe_v1024_en"
        type: bpe
      de: # German tokenizer (example, replace with whichever language you would like or add tokenizers to add tokenizer for additional languages)
        dir: "tokenizers/spe_bpe_v1024_en"
        type: bpe
```

## 2. Training on a new task: A case of decoding with context

This is an example of a capability that is already supported by the current [Canary2PromptFormatter](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/common/prompts/canary2.py) as well as the tokenizer model.

```
"decodercontext": Modality.Text
```

During training, you will pass an additional `decodercontext` argument to the samples in the manifest.
```
metadata = {
    "audio_filepath": audio_path,
    "duration": duration,
    "text": transcript,
    "target_lane": "en",
    "source_lang": "en",
    "decodercontext": decoder_context,
}
```

For example, the `decodercontext` can represent past context or certain keywords or topic of the spoken content. The current implementation assumes that `decodercontext` and the output transcript have the same language.

## 3. Training on a new task: A case of timestamp prediction

Canary-Flash models support timestamp prediction. Here, we include how the manifest, prompt formatter, special tokens, and tokenizer functions were modified to add timestamps support for the Canary model.

Canary-Flash interleaves word-level timestamps as frame numbers before and after the word. These the frame numbers correspond to the start and end of a word segment. Such "interleaving" patterns might be relavant for other tasks as well such as multi-speaker recognition, where you want to interleave speaker ID tokens before appropriate chunks of text tokens spoken by that speaker.

Below, we show how a sample in manifest changes with and without timestamps:

```
# without timestamps
metadata = {
    "audio_filepath": audio_path,
    "duration": duration,
    "text": "it's almost beyond conjecture",
    "target_lane": "en",
    "source_lang": "en",
    "timestamp": "no",
}
```

```
# with timestamps
metadata = {
    "audio_filepath": audio_path,
    "duration": duration,
    "text": "<|3|> it's <|7|> <|8|> almost <|9|> <|14|> beyond <|20|> <|20|> conjecture <|28|>",
    "target_lane": "en",
    "source_lang": "en",
    "timestamp": "yes",
}
```

In order to support this functionality, the [Canary2PromptFormatter](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/common/prompts/canary2.py) should have the relevant slot value and the default values:

```
# Should we predict timestamps?
"timestamp": Modality.TextLiteral(
    "yes",
    "no",
    "true",
    "True",
    "false",
    "False",
    "1",
    "0",
    "timestamp",
    "notimestamp",
    "<|timestamp|>",
    "<|notimestamp|>",
),
```

The default can be set as `<|notimestamp|>`:
```
optional_slots = {
    "decodercontext": "",
    "emotion": "<|emo:undefined|>",
    "itn": "<|noitn|>",
    "timestamp": "<|notimestamp|>",
    "diarize": "<|nodiarize|>",
    "pnc": "<|pnc|>",  
}
```

Additionally we need tokens to support these additional task-related tokens, `<|timestamp|>`, `<|notimestamp|>`, and integer tokens to encode frame indices.
We add 900 integers to the list special tokens along with task-related tokens and rebuild the tokenizer as previously discussed.

Now the transcript is a mix of tokens from `spl_tokens` tokenizer (frame indices) and tokens from a language-specific tokenizer.
The [canary_tokenizer](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/common/tokenizers/canary_tokenizer.py) handles this by adding a modified `_text_to_ids` method.


```
def _text_to_ids_maybe_with_timestamps(self, text_no_eos, lang_id) -> list[int]:
    time_pattern = re.compile(r"<\|\d+\|>")
    time_text = "".join(time_pattern.findall(text_no_eos))
    has_timestamp = bool(time_text)
    if not has_timestamp:
        return super().text_to_ids(text_no_eos, lang_id)
    else:
        text_without_timestamps = time_pattern.sub("", text_no_eos).strip()
        return self._text_with_timestamps_to_ids(text_without_timestamps, time_text, lang_id)

```

Once these changes are in place, you should be able to train the model on data with word-level timestamps.

## 4. Training on a new task: A case of speech summarization

Speech summarization is an example of completely new task, meaning, neither the prompt format nor the default special tokens have an explicit support for this task.

You will start with modifying [Canary2PromptFormatter](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/common/prompts/canary2.py) or even writing your own custom prompt formatter. [This tutorial](https://github.com/NVIDIA/NeMo/blob/main/tutorials/multimodal/Prompt%20Formatter%20Tutorial.ipynb) has useful references on modifying and building custom prompt formatter.


One possible way to modify the existing promp format is to add an optional `"summarize"` key whose default value is `false`:
```
# should we summarize?
"summarize": Modality.TextLiteral(
    "yes",
    "no",
    "true",
    "True",
    "false",
    "False",
    "1",
    "0",
    "<|summarize|>",
    "<|nosummarize|>"
),
```

The default can be set as `<|nosummarize|>`:
```
optional_slots = {
    "decodercontext": "",
    "emotion": "<|emo:undefined|>",
    "itn": "<|noitn|>",
    "timestamp": "<|notimestamp|>",
    "diarize": "<|nodiarize|>",
    "pnc": "<|pnc|>",  
    "summarize": "<|nosummarize|>",
}
```
Then, you'll pass `"summarize": true` to the manifest for samples from speech summarization data, where the corresponding `text` will refer to the summary text.

```
metadata = {
    "audio_filepath": audio_path,
    "duration": duration,
    "text": summary, # note that this is now a text summary and not a transcript
    "target_lane": "en",
    "source_lang": "en",
    "summarize": "true",
}
```

The default list of special tokens does not have `<|summarize|>` and `<|nosummarize|>` in the vocabulary. So you'll want to build a new tokenizer for the new vocabulary of `spl_tokens`.

You can selectively retain token embeddings for the matched tokens, or simply reinitialize all token embeddings.


## 5. Starting from Canary-flash checkpoint

For any of the above scenarios, you may choose to intialize the model from one of the public Canary-flash checkpoints. In the previous section we saw a working example of fine-tuning from a Canary-flash checkpoint. Here we see how we can customize the arguments.

We use the `include` and `exclude` paramaters to appropriately restore or drop certain weights, in case there is a difference in tokenizer or model architecture.


  (i) Initialize all the parameters

  ```
  init_from_pretrained_model:
    model0:
      name: "nvidia/canary-180m-flash"
  ```

  (ii) Initialize just the encoder:
  ```
  init_from_pretrained_model:
    model0:
      name: "nvidia/canary-180m-flash"
      include: ["encoder"]
  ```

  (iii) Initialize encoder and decoder but not the token embeddings (relevant for scenarios that use a different tokenizer):
  ```
  init_from_pretrained_model:
    model0:
      name: "nvidia/canary-180m-flash"
      exclude: ["transf_decoder._embedding.token_embedding", "log_softmax.mlp.layer0"]

  ```

  (iv) If you wish further customization that cannot be handled with just these arguments, you can modify https://github.com/NVIDIA/NeMo/blob/main/nemo/core/classes/modelPT.py. Specifically, modify the following snippet of code

  ```
  dict_to_load = {}
  for k, v in state_dict.items():
      should_add = False
      # if any string in include is present, should add
      for p in include:
          if p in k:
              should_add = True
              break
      # except for if any string from exclude is present
      for e in exclude:
          if e in k:
              excluded_param_names.append(k)
              should_add = False
              break
      if should_add:
          dict_to_load[k] = v
  ```

# Practitioner's tips

## Starting from a pre-trained checkpoint

In our experience working with Canary, we noticed that starting from a pre-trained speech encoder, greatly helps convergence. Especially for larger models (1B+ params) initializing from a pretrained encoder may even be required to stabilize the training.

Canary-180M-Flash 17-layer fastconformer encoder was initialized from a 17-layer fastconformer encoder of a transducer speech recognition model ([model](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/nemo/models/stt_multilingual_fastconformer_hybrid_large_pc_blend_eu/files), [config](https://github.com/NVIDIA/NeMo/blob/main/examples/asr/conf/fastconformer/fast-conformer_transducer_bpe.yaml#L29)). The 4-layer transformer decoder was initialized from scratch.

Canary-1B-Flash has 32-layer fastconformer encoder. The first 24 layers were initialized from a 24-layer fastconfromer encoder of a transducer speech recognition model and the rest were randomly initalized. This 24-layer model was training internally with this [config](https://github.com/NVIDIA/NeMo/blob/main/examples/asr/conf/fastconformer/fast-conformer_transducer_bpe.yaml#L31).

## Training Canary for multiple tasks

We have seen that Canary-Flash models support multiple capabilities -- speech recognition in four languages (ASR), speech to text translation (AST)for six language pairs, timestamp (TS) prediction in four languages. Canary-Flash models are also optimized to be robust to background noise (NR) and hallucination (HR).

These capabilties were achieved over three stages of training:
* **Stage 1**: ASR+AST
* **Stage 2**: ASR+AST+HR+NR
* **Stage 3**: ASR+AST+HR+NR+TS

At each stage, we add new capability to the model and at the same time we continue supervised training for previously learned capabilities. This is essential for the model to learn without forgetting.

So, whenever you perform Canary-style training, irrespective of whether or not you start from a Canary-Flash checkpoint, make sure that the training data mix includes supervision for all the capabilities (tasks and languages) that you wish the final model to learn and retain.  

## Training efficiency with 2-D bucketing and OOMptimizer

Canary-Flash training is also optimized for optimal GPU utilization. 2-D bucketing and OOMptimizer are the two key components of for optimal GPU utilization, handled by the config as shown below.
```
model:
  train_ds:
    use_bucketing: true
    bucket_duration_bins: [[3.79,27],[3.79,65],[4.8,34],[4.8,66],[5.736,39],[5.736,73],[6.42,44],[6.42,79],[7.182,47],[7.182,87],[8.107,52],[8.107,100],[8.78,60],[8.78,111],[9.62,66],[9.62,115],[10.47,71],[10.47,127],[11.14,76],[11.14,139],[11.8,78],[11.8,139],[12.47,82],[12.47,150],[13.02,88],[13.02,160],[13.55,92],[13.55,160],[14.1,94],[14.1,168],[14.64,97],[14.64,169],[15.15,101],[15.15,175],[15.63,102],[15.63,170],[16.09,104],[16.09,180],[16.63,107],[16.63,186],[17.17,109],[17.17,184],[17.71,113],[17.71,206],[18.18,116],[18.18,208],[18.67,119],[18.67,209],[19.13,123],[19.13,210],[19.61,125],[19.61,226],[20.18,126],[20.18,232],[32.467,184],[32.467,321],[36.567,243],[36.567,398],[40.0,272],[40.0,437]]
    bucket_batch_size: [334,314,264,248,221,214,196,190,174,169,155,146,142,134,126,123,116,112,106,103,103,95,95,92,92,89,89,86,84,82,80,78,78,76,76,74,74,72,72,68,68,66,66,64,64,62,62,60,60,58,58,56,56,54,33,32,29,28,26,25]
```

See these parameters for `canary-180m-flash` model:

In [ ]:
# Load canary model if not previously loaded in this notebook instance
if 'canary_model' not in locals():
    canary_model = EncDecMultiTaskModel.from_pretrained('nvidia/canary-180m-flash')

In [ ]:
print('bucket_duration_bins: \n', canary_model._cfg['train_ds']['bucket_duration_bins'])
print('bucket_batch_size: \n', canary_model._cfg['train_ds']['bucket_batch_size'])

Simply put, these tools set the optimal batch statistics after considering the distribution of lengths of input audio, lengths of decoder outputs (decoder prompt and tokenized transcript), and model size. Bucketing (`bucket_duration_bins`)ensures that a training batch does not have samples of uneven lengths, as that would lead to wasteful usage of memory by the `<pad>` tokens. OOMptimizer sets batchsizes (`bucket_batch_sizes`) for each bucket ensuring that the training utilizes optimal GPU memory while not running into OOM errors.



An alternative, if you don't wish to use bucketing, is to set the batchsize explicitly.
```
model:
  train_ds:
    use_bucketing: false
    batch_size: 32
```

Next we add pointers to the script that compute `bucket_duration_bins` and `bucket_batch_sizes`. You will need config for your data, config for your model, and paths to tokenizers.

Let's say `$NEMO_DIR` is path to the installed NeMo library.

First step is to estimate 2D buckets bins using the data config and tokenizers. It takes as arguments, number of buckets, number of sub-buckets (2D in our case), number of utterances used to estimate the bins, lowest and highest duration in seconds, and arguments related to dataset manifest, tokenizers, and prompt format.

```
python $NEMO_DIR/scripts/speech_recognition/estimate_duration_bins_2d.py \
    -b 30 \
    -s 2 \
    -n 100000 \
    -l 0.5 -u 40.0 \
    -t $tokenizer_model1 $tokenizer_model2 $tokenizer_model3 \
    -a $lang1 $lang2 $lang3 \
    --lang-field target_lang \
    --text-field answer \
    -f canary2 \
    -p "[{'role':'user','slots':{'source_lang':'en','target_lang':'en','pnc':'yes','decodercontext':'','emotion':'<|emo:undefined|>','itn':'yes','diarize':'yes','timestamp':'yes'}}]" \
    $dataset_config
```

The next step is to obtain `bucket_batch_sizes` using the estimated `bucket_duration_bins` and model config.
```
BUCKETS=$bucket_duration_bins

python $NEMO_DIR/scripts/speech_recognition/oomptimizer.py \
    -m nemo.collections.asr.models.EncDecMultiTaskModel\
    -c $config \
    --no-ddp \
    -b "$BUCKETS"

```

Then you'd update the training config accordingly and launch a training job as shown before.

If you are interested to learn more about these tools, we discuss illustrative examples, technical details, and report efficiency gains in [Zelasko et al.](https://arxiv.org/abs/2503.05931).

Refer to documentation on [2-D bucketing](https://docs.nvidia.com/nemo-framework/user-guide/latest/nemotoolkit/asr/datasets.html#d-bucketing) and [OOMptimizer](https://docs.nvidia.com/nemo-framework/user-guide/latest/nemotoolkit/asr/datasets.html#pushing-gpu-utilization-to-the-limits-with-bucketing-and-oomptimizer) for more details.

## Masking loss for prompt tokens

The config has `use_loss_mask_for_prompt` parameter which decides whether or not the training objective includes loss for the decoder prompt tokens.

We noticed that masking prompt loss tokens led to a better performing `canary-180m-flash` model, where as it did not make any noticeable difference for `canary-1b-flash`.

In [ ]:
# Load canary model if not previously loaded in this notebook instance
if 'canary_model' not in locals():
    canary_model = EncDecMultiTaskModel.from_pretrained('nvidia/canary-180m-flash')

In [ ]:
print('prompt loss masking for canary-180m-flash: \n', canary_model._cfg['use_loss_mask_for_prompt'])

# Follow-up reading material and tutorials

1. [SentencePiece](https://arxiv.org/abs/1808.06226) and [concatenated](https://arxiv.org/abs/2306.08753) tokenizer: To learn more about the tokenization process.


2. [Tutorial on prompt formatter](https://github.com/NVIDIA/NeMo/blob/main/tutorials/multimodal/Prompt%20Formatter%20Tutorial.ipynb): To learn more about prompt formatter.

2. [Tutorial on multi-task adapters](https://github.com/NVIDIA/NeMo/blob/main/tutorials/asr/asr_adapters/Multi_Task_Adapters.ipynb): If you wish to explore adaptation of `Canary-flash` checkpoints using adapters.